In [1]:
from experiment import Experiment
from utils.behavior_data import BehaviorData
from visuals import Plotter
import torch
import numpy as np
from utils.state_data import StateData
import matplotlib.pyplot as plt

In [ ]:
for respond_perc in [.5]:
# for respond_perc in [.75, .25]:
    # for estate, include_state in [(True, True)]:
    for estate, include_state, fullq in [(False, True, False)]:
        for model in ["AdaptableLSTM", "BasicNN"]:
            for learning_rate in [.012]:
                test_metrics, train_metrics, adjusted_losses = [], [], []
                epochs = 90
                if (model != "AdaptableLSTM"):
                    epochs = 75
                    learning_rate *= .2
                for seed in range(1):
                    np.random.seed(seed)
                    torch.manual_seed(seed)
                    e = Experiment(
                        numValFolds = 5,
                        data_kw={"minw": 2,
                                "maxw": 29,
                                "include_state": include_state,
                                "include_pid": False,
                                "expanded_states": estate,
                                "top_respond_perc": respond_perc,
                                 "full_questionnaire": fullq
                                },
                        model=model,
                        model_kw={
                            "lossfn": "MSELoss",
                            # "lossfn": "NDCG",
                            # "lossfn": "CrossEntropyLoss",
                            "hidden_size": 200, 
                            "opt_kw": {
                                "lr": learning_rate
                            }
                        },
                        train_kw={
                            "epochs": epochs,
                            "n_subj": 500,
                            "rec_every": 5,
                        })
                    print(e.model)
                    report = e.run()

                    feature_grads = e.calcFinalGradients()
                    print(feature_grads / feature_grads.abs().max())
                    print(np.argsort(feature_grads))



268 537
(6904, 38) (6904, 8)
AdaptableLSTM(
  (lstm): LSTM(38, 200)
  (fc_q1): Linear(in_features=200, out_features=3, bias=True)
  (fc_q2): Linear(in_features=200, out_features=3, bias=True)
  (relu): ReLU()
)
0	 [0.22435282170772552]
5	 [0.20647960901260376]
10	 [0.2045901119709015]
15	 [0.1998584121465683]
20	 [0.1918262392282486]
25	 [0.18681631982326508]
30	 [0.1856691986322403]
35	 [0.17988725006580353]
40	 [0.17629101872444153]
45	 [0.17149241268634796]
50	 [0.16778986155986786]
55	 [0.16276471316814423]
60	 [0.16011574864387512]
65	 [0.15014542639255524]
70	 [0.1476132869720459]
75	 [0.1360962688922882]
80	 [0.1238156333565712]
85	 [0.11685342341661453]
89	 [0.11089825630187988]
tensor([-4.6630e-02,  2.8902e-02, -3.0769e-02, -6.1676e-02, -7.5389e-02,
        -1.2693e-01, -2.0562e-01,  3.0873e-02,  4.3846e-03, -9.3837e-02,
        -2.8079e-01,  4.9207e-01, -6.8771e-01, -3.0493e-01,  1.7273e-01,
        -7.2129e-02,  9.1850e-04, -1.6923e-01, -2.3735e-01, -1.7226e-01,
         3.5

In [ ]:
matplotlib.pyplot(feature_grads / feature_grads.abs().max()